In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Squeeze-and-Excitation (SE) Module
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)  # Global Average Pooling
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)
    
    def forward(self, x):
        batch, channels, _ = x.shape
        se = self.global_avg_pool(x).view(batch, channels)  # Global Avg Pool
        se = F.relu(self.fc1(se))  # First FC + ReLU
        se = F.softmax(self.fc2(se), dim=1)  # Second FC + Sigmoid
        se = se.view(batch, channels, 1)
        return x * se  # Scale the input

# Residual Block with SE Module
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=kernel_size//2)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=kernel_size//2)
        self.se = SEBlock(out_channels)
        self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else nn.Identity()
        self.maxpool = nn.MaxPool1d(kernel_size=2)  # Add MaxPooling layer

    
    def forward(self, x):
        x= self.maxpool(x)  # Apply MaxPooling
        residual = self.shortcut(x)  # Skip connection
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.conv2(out)
        out = self.se(out)  # Apply SE Module
        out += residual  # Add Skip Connection
        return F.relu(out)

# Full Model: CNN + BiLSTM + FC
class ECGClassifier(nn.Module):
    def __init__(self, num_classes=5):
        super(ECGClassifier, self).__init__()

        # First Conv -> ReLU -> BatchNorm
        x = self.conv1(x)
        x = F.relu(x)
        x = self.bn1(x)

        # Second Conv -> ReLU -> BatchNorm
        x = self.conv2(x)
        x = F.relu(x)
        x = self.bn2(x)

        # Residual Blocks with SE Module (4 Blocks)
        self.resblock1 = ResidualBlock(32, 64)
        self.resblock2 = ResidualBlock(64, 96)
        self.resblock3 = ResidualBlock(96, 128)
        self.resblock4 = ResidualBlock(128, 160)

        # MaxPooling Layer
        self.maxpool = nn.MaxPool1d(kernel_size=2)

        # BiLSTM Layers
        self.lstm = nn.LSTM(160, 64, num_layers=2, bidirectional=True, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.5)

        # Fully Connected Layers
        self.fc1 = nn.Linear(128, 64)  # BiLSTM outputs 64*2=128
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        # CNN Feature Extraction
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.bn2(F.relu(self.conv2(x)))

        # Residual Blocks with SE Module
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.resblock3(x)
        x = self.resblock4(x)

        # Max Pooling
        x = self.maxpool(x)

        # Reshape for LSTM (Batch, SeqLen, Features)
        x = x.permute(0, 2, 1)

        # BiLSTM
        x, _ = self.lstm(x)
        x = self.dropout1(x[:, -1, :])  # Take last LSTM output
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)  # Softmax for classification

# Create Model Instance
model = ECGClassifier(num_classes=5)

print(model)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/mpallasmichael/Desktop/ecgDiagnosis/venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/mpallasmichael/Desktop/ecgDiagnosis/venv/lib/python3.9/sit

ECGClassifier(
  (conv1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (resblock1): ResidualBlock(
    (conv1): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,))
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,))
    (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (se): SEBlock(
      (global_avg_pool): AdaptiveAvgPool1d(output_size=1)
      (fc1): Linear(in_features=64, out_features=4, bias=True)
      (fc2): Linear(in_features=4, out_features=64, bias=True)
    )
    (shortcut): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
  )
  (resblock2): ResidualBlock(
   